# Loan Cancellation #

## Summary
I focused on building a parsimonious model by first reducing the number of variables to a workable representative set, and then understanding the effect of each variable on the probability of loan cancellation, and discarding variables with limited predictivity. My final model uses whether the loanee has taken out a Prosper loan before, the size of loan requested and the loan's interest rate, as well as loanee's monthly debt, monthly income, employment status, credit card use and number of real estate trades, and the number of loan inquiries initialized as predictors. My final model achieves an AUC score of xxx.

Insert Figure

## Methodology

### Dimensional Reduction

To handle the large number of columns in the data (86 to start), my initial focus was dimensional reduction: Finding a smaller set of variables that still encompasses most of the original information. One way to do this would be Principal Component Analysis (PCA) followed by rotation of eigenvectors, but since I was dealing with an unfamiliar data set, I preferred to use a different method to allow me to pare down data more deliberately. Instead, I used Pearson's r to find variables that were highly correlated, which I then turned into a graph with networkx to find connected families. After doing this, I chose one representive variable from each family and dropped the rest. 

My first application of this graph technique with r = .55 found twelve families and reduced the number of variables from 80 to 56. I chose an r such that the max family size would be no larger than 10. I then reapplied the same method with a lower cutoff of r = .45. This method proved to be imperfect - in the original run-through, I ended up discarding NumPriorProsperLoans, which I later realized to be highly important.

### Logistic Regression with Individual Variables

I then performed a logistic regression with each individual variable to see whether a significant relation exists, and then I started to build my final logistic model one variable at a time. In the case of clearly related families of variables, such as the combination of DolMonthlyDebt-DolMonthlyIncome-FracDebtToIncomeRatio, I investigated the set of variables as a group to see what combination of variables provided better predictivity (in this case, DolMonthlyDebt and DolMonthlyIncome performed best). Plugging in each variable at a time proved painstaking, but given my unfamiliarity with loan data I didn't feel comfortable with making assumptions about which variables might be significant.

In the end, I created the variable BoolPriorProsperLoanee, used the numerical variables DolLoanAmountRequested, PctBankcardUtil, DolMonthlyDebt, DolMonthlyIncome, BorrowerRate, NumRealEstateTrades, StrEmploymentStatus, NumTotalInquiries' and the categorical variable StrEmploymentStatus.

## Findings

* The single best predictor of loan cancellation is simply familiarity with the Prosper platform. Users who have taken out a Prosper loan before have only a 13% chance of cancellation, versus 36% if they have not.

* Potential loanees who list their employment status as "Other" are much more likely to cancel than ...

* One might expect the interest rate of the loan to strongly influence cancellations, but in fact interest rate has only a subtle influence on cancellation, subordinate to other factors.

In [1]:
from datetime import datetime, timedelta
import itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import scipy.stats as stats
import seaborn as sns
from sklearn import linear_model
from sklearn import metrics
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
from sklearn.pipeline import make_pipeline

import networkx as nx
from networkx.algorithms.components.connected import connected_components

%matplotlib notebook
%timeit

#show up to 100 columns and rows.
pd.set_option('display.max_columns', 100, 'display.max_rows', 100)

Load in the data from the pickle file

In [2]:
data_file = "new_theorem_data.p"

## data appears to have been savied in Python 2 - changing encoding allows us to properly load data
with open(data_file, 'rb') as pickle_file:
    data = pickle.load(pickle_file, encoding='latin1') 

Sample listing below

In [3]:
data.iloc[0]

ListingID                                                       973605
DateCreditPulled                                   2013-10-13 01:50:58
DateListingStart                            2014-01-26 19:00:08.887000
DateListingCreation                         2013-10-13 01:50:56.287000
EnumListingStatus                                                    7
DolLoanAmountRequested                                           15000
BoolPartialFundingApproved                                        True
CreditGrade                                                          B
LenderYield                                                      0.152
BorrowerRate                                                     0.162
NumMonthsTerm                                                       60
DolMonthlyLoanPayment                                           366.37
FICOScore                                                          689
ProsperScore                                                         6
EnumLi

In [4]:
print('Number of rows:',len(data))
cols = data.columns.values
print('Number of columns:',len(cols))

Number of rows: 252469
Number of columns: 86


Let's visualize how the different columns are inter-related by showing a heat map of Pearson's r between pairs of variables.

In [5]:
corr = data.corr()
corr.head()

## 86 different categories - start by taking correlation matrix to figure out which categories are redundant
f, ax = plt.subplots(figsize=(9,7))

# Draw the heatmap using seaborn
sns.heatmap(corr, square=True, cbar=True, xticklabels=False, yticklabels=False)

<IPython.core.display.Javascript object>

There are many visible families of variables encoding very similar information. Let's gather these sets of variables by turning them into a graph network. I've provided a solution using networkx below.

In [6]:
## thank you to stack overflow for the elegant solution: 
## http://stackoverflow.com/questions/4842613/merge-lists-that-share-common-elements
def to_graph(l):
    G = nx.Graph()
    for part in l:
        G.add_nodes_from(part)
        G.add_edges_from(to_edges(part))
    return G

def to_edges(l):
    """ 
        treat `l` as a Graph and returns it's edges 
        to_edges(['a','b','c','d']) -> [(a,b), (b,c),(c,d)]
    """
    it = iter(l)
    last = next(it)

    for current in it:
        yield last, current
        last = current    

The following function outputs a sorted list of all families of highly-correlated variables in a matrix. Takes in a dataframe of correlations and outputs a list of lists.

In [7]:
def find_families(corr_df, thresh):
    
    tentative_families = []

    ## for each variable, find each other variable that is highly correlated with, and put into list form.
    for name, col in corr_df.iteritems():

        highly_correlated = (abs(col) > thresh) & (col.index != name)
        high_corr_list = col[highly_correlated].index.tolist()

        #need to add the variable itself if the family is non-empty
        if high_corr_list != []:
            tentative_families.append(sorted(high_corr_list + [ name ]))

    tentative_families.sort()
    tentative_families = list(tentative_families for tentative_families,_ in itertools.groupby(tentative_families))
        
    ## use networkx class to convert list of connected nodes into a graph
    G = to_graph(tentative_families)
    cc = nx.connected_components(G)  ## returns a list of connected nodel elements
    families = []

    for nodes in cc:
        families.append(list(nodes))

    families = sorted([ sorted(fam) for fam in families ])

    ## let's print a list of all the families we've found
    for fam in families:
        print(fam, len(fam))
   
    ll = [ len(fam) for fam in families ]
    print("In total,", sum(ll), "variables found in", len(families), "families.")
    
    return families

For each variable in our data, we find a list of variables that it is highly correlated with, transform these linkages into a graph, and output families of connected nodes.

In [8]:
## cutoff of .6 recovers 47 inter-connected variables, .5 recovers 52, .55 recovers 50
## cutoff of .55 features largest group at length 10 - probably don't want to exceed that
cutoff = .55
families_1 = find_families(corr,cutoff)

['BoolEverWholeLoan', 'BoolIsFractionalLoan', 'EnumLoanFractionalType'] 3
['BoolOwnsHome', 'DolMonthlyDebt', 'DolRealEstateBalance', 'DolRealEstatePayment', 'DolTotalBalanceAllOpenTrades6', 'DolTotalBalanceInstallTradesReptd6', 'DolTotalPaymentAllOpenTrades6'] 7
['BorrowerRate', 'LenderYield', 'ProsperScore'] 3
['DolLoanAmountRequested', 'DolMonthlyLoanPayment'] 2
['DolMaxPriorProsperLoan', 'DolMinPriorProsperLoan', 'DolPriorProsperLoansBalanceOutstanding', 'DolPriorProsperLoansPrincipalBorrowed', 'DolPriorProsperLoansPrincipalOutstanding', 'NumPriorProsperLoans', 'NumPriorProsperLoansActive', 'NumPriorProsperLoansCyclesBilled', 'NumPriorProsperLoansEarliestPayOff', 'NumPriorProsperLoansOnTimePayments'] 10
['DolRevolvingBalance', 'DolTotalBalanceOpenRevolving6'] 2
['NumBankcardTradesOpened12', 'NumCreditLines84', 'NumCurrentCreditLines', 'NumOpenCreditLines', 'NumOpenRevolvingAccounts', 'NumSatisfactoryAccounts', 'NumTrades'] 7
['NumDelinquencies84', 'NumDelinquenciesOver30Days', 'NumD

I go through each family individually, give each one a name, and then choose a single variable to represent that family. Then, drop all of the variables in each family except for the chosen representative variable.

In [9]:
## give custom descriptive names to each family
# two items: first, each unit in the family; second, the variable(s) you're going to keep around.
family_dict = {}
family_dict['fractional_loan'] = [families_1[0],'BoolEverWholeLoan']
family_dict['debt'] = [families_1[1],'DolMonthlyDebt']
family_dict['borrower_rate'] = [families_1[2],'BorrowerRate']
family_dict['loan_amount'] = [families_1[3],'DolLoanAmountRequested']
family_dict['prosper_history'] = [families_1[4],'NumPriorProsperLoans']
family_dict['revolving_balance'] = [families_1[5],'DolTotalBalanceOpenRevolving6']
family_dict['credit'] = [families_1[6],'NumCurrentCreditLines']
family_dict['delinquencies'] = [families_1[7],'PctTradesNeverDelinquent']
family_dict['inquiries'] = [families_1[8],'NumTotalInquiries']
family_dict['prior_prosper_loans'] = [families_1[9],'NumPriorProsperLoans61dpd']
family_dict['real_estate'] = [families_1[10],'NumRealEstateTrades']
family_dict['current_delinquency'] = [families_1[11],'NumTradesCurr30DPDOrDerog6']

## leave original data unaltered, work with dataframe reduced_data instead...
reduced_data = pd.DataFrame.copy(data, deep = True)

In [10]:
## drop similar columns as determined by families
for fam, items in family_dict.items():
    drop_cols = list(items[0])
    drop_cols.remove(items[1]) #keep our chosen representative variable for that family
    reduced_data.drop(drop_cols, axis = 1, inplace = True)
    
print('New number of columns after first filtering:',len(reduced_data.columns.values))

## save this preliminary set of variables as a pickle file.
reduced_data.to_pickle('theorem_reduced_firstfilter.pkl')

New number of columns after first filtering: 48


Can take a quick look at remaining columns.

In [11]:
print(reduced_data.columns.values)

['ListingID' 'DateCreditPulled' 'DateListingStart' 'DateListingCreation'
 'EnumListingStatus' 'DolLoanAmountRequested' 'BoolPartialFundingApproved'
 'CreditGrade' 'BorrowerRate' 'NumMonthsTerm' 'FICOScore'
 'EnumListingCategory' 'DolMonthlyIncome' 'BoolIncomeVerifiable'
 'FracDebtToIncomeRatio' 'StrEmploymentStatus' 'StrOccupation'
 'NumMonthsEmployed' 'StrState' 'StrBorrowerCity' 'NumPriorProsperLoans'
 'NumPriorProsperLoansLateCycles' 'NumPriorProsperLoansLatePayments'
 'NumPriorProsperLoans61dpd' 'BoolIsLender' 'BoolInGroup' 'EnumChannelCode'
 'NumTradesOpened6' 'NumOpenTradesDelinqOrPastDue6'
 'NumTradesCurr30DPDOrDerog6' 'DolTotalBalanceOnPublicRecords'
 'AgeOldestTrade' 'PctTradesNeverDelinquent' 'DolTotalAvailBankcardCredit6'
 'NumRealEstateTrades' 'DolTotalBalanceOpenRevolving6' 'DolMonthlyDebt'
 'NumCurrentDelinquencies' 'NumPublicRecordsLast10Years'
 'NumPublicRecords12' 'DateFirstCredit' 'DolAmountDelinquent'
 'NumCurrentCreditLines' 'PctBankcardUtil' 'NumTotalInquiries'
 'D

Start looking at individual columns and remove extraneous columns such as dates and ID numbers. For instance, variables such as DateCreditPulled, DateListingCreation, DateListingStart and DateWholeLoanStart are unlikely to reflect consumer decision-making. Similarly, while there may be interesting things to be said about the influence of state/city on loan cancellation, they again shouldn't really strongly affect decision-making, and any effects are probably subordinate to direct economic effects.

In [12]:
## immediately drop columns that are unlikely/unable to cause loan cancellation (ID numbers/dates)
drop_cols = ['ListingID', 'DateCreditPulled', 'DateListingStart', 'DateListingCreation', 'DateWholeLoanStart', 'DateWholeLoanEnd']
reduced_data.drop(drop_cols, axis = 1, inplace = True)

## manually drop a few other columns that won't help this analysis.
reduced_data.drop(['StrState','StrBorrowerCity'], axis=1, inplace=True)

Although it's still too early to work with each column individual, some of these columns can be translated into more useful form - in particular, DateFirstCredit.

In [13]:
## replace DateFirstCredit column with an integer
days_since_firstcredit = datetime.now()-reduced_data['DateFirstCredit']
reduced_data['DaysSinceFirstCredit'] = [ x.days for x in days_since_firstcredit ]
reduced_data.drop('DateFirstCredit', axis=1, inplace=True)

Similarly, it turns out that CreditGrade encodes the same information as BorrowerRate, except with greater precision.

In [14]:
## quick plot to show that BorrowerRate and CreditGrade are pretty much equivalent - drop CreditGrade
reduced_data['BorrowerRate'].hist(by=reduced_data['CreditGrade'])
reduced_data.drop('CreditGrade',axis =1, inplace=True)

<IPython.core.display.Javascript object>

We're down to 39 columns.

In [15]:
print('New number of columns after removing variables:',len(reduced_data.columns.values))

New number of columns after removing variables: 39


Let's use the graph-family-correlation method again to further reduce the number of variables we're working with, this time with a lower threshold.

In [16]:
## show another correlation matrix, this time with reduced set of variables.
reduced_corr = reduced_data.corr()

f, ax = plt.subplots(figsize=(9,7))

# Draw the heatmap using seaborn
sns.heatmap(reduced_corr, square=True, cbar=True, xticklabels=False, yticklabels=False)

<IPython.core.display.Javascript object>

Still pockets of highly-correlated variables present, so let's rerun the method.

In [17]:
cutoff = .45
families_2 = find_families(reduced_corr,cutoff)

['AgeOldestTrade', 'DaysSinceFirstCredit'] 2
['BorrowerRate', 'DolTotalAvailBankcardCredit6', 'FICOScore', 'PctTradesNeverDelinquent'] 4
['DolMonthlyDebt', 'DolTotalBalanceOpenRevolving6', 'NumCurrentCreditLines'] 3
['NumCurrentDelinquencies', 'NumTradesCurr30DPDOrDerog6'] 2
['NumPriorProsperLoansLateCycles', 'NumPriorProsperLoansLatePayments'] 2
In total, 13 variables found in 5 families.


In [18]:
## give custom descriptive names to each family
# two items: first, each unit in the family; second, the variable(s) you're going to keep around.
family_dict_2 = {}
family_dict_2['oldest_trade'] = [families_2[0],'DaysSinceFirstCredit']
family_dict_2['borrower_rate'] = [families_2[1],'BorrowerRate']
family_dict_2['monthly_debt'] = [families_2[2],'DolMonthlyDebt']
family_dict_2['current_delinquencies'] = [families_2[3],'NumCurrentDelinquencies']
family_dict_2['prosper_delinquencies'] = [families_2[4],'NumPriorProsperLoansLatePayments']

## again, drop similar columns as determined by families
for fam, items in family_dict_2.items():
    drop_cols = list(items[0])
    drop_cols.remove(items[1]) #keep our chosen representative variable for that family
    reduced_data.drop(drop_cols, axis = 1, inplace = True)

In [19]:
print("Number of columns:",len(reduced_data.columns.values))
reduced_data.head()

Number of columns: 31


,EnumListingStatus,DolLoanAmountRequested,BoolPartialFundingApproved,BorrowerRate,NumMonthsTerm,EnumListingCategory,DolMonthlyIncome,BoolIncomeVerifiable,FracDebtToIncomeRatio,StrEmploymentStatus,StrOccupation,NumMonthsEmployed,NumPriorProsperLoans,NumPriorProsperLoansLatePayments,NumPriorProsperLoans61dpd,BoolIsLender,BoolInGroup,EnumChannelCode,NumTradesOpened6,NumOpenTradesDelinqOrPastDue6,DolTotalBalanceOnPublicRecords,NumRealEstateTrades,DolMonthlyDebt,NumCurrentDelinquencies,NumPublicRecordsLast10Years,NumPublicRecords12,DolAmountDelinquent,PctBankcardUtil,NumTotalInquiries,BoolEverWholeLoan,DaysSinceFirstCredit
ListingNumber,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
973605,7,15000.0,True,0.1620,60,1,6000.0000,True,0.27,Employed,Tradesman - Mechanic,445.0,0,NaN,NaN,0,False,70000,1,0,0,2,1242,0,0,0,0,0.97,5,True,13907
981099,7,15000.0,True,0.1585,60,1,7916.6667,True,0.35,Other,NaN,32.0,0,NaN,NaN,0,False,70000,1,0,0,2,2289,0,0,0,0,0.48,3,True,14244
1025766,6,4000.0,True,0.2085,36,1,2083.3333,True,0.53,Employed,Professional,4.0,3,0.0,NaN,0,False,80000,0,0,0,0,911,0,0,0,0,0.93,5,False,4152
1003835,7,10000.0,True,0.1299,36,13,3750.0000,True,0.14,Employed,Medical Technician,2.0,0,NaN,NaN,0,False,90000,1,0,0,0,223,0,0,0,0,0.26,1,True,2948
1011335,6,20000.0,True,0.1440,60,1,9000.0000,True,0.16,Employed,Executive,90.0,1,0.0,NaN,0,False,80000,1,0,1249,1,1264,1,2,0,0,0.81,17,False,8335


A closer look at BoolIncomeVerifiable shows that it is completely equivalent to StrEmploymentStatus - can safely drop.

In [20]:
print('BoolIncomeVerifiable is false:\n', \
      reduced_data[reduced_data['BoolIncomeVerifiable'] == False]['StrEmploymentStatus'].value_counts(normalize = True))
print('BoolIncomeVerifiable is true:\n', \
      reduced_data[reduced_data['BoolIncomeVerifiable'] == True]['StrEmploymentStatus'].value_counts(normalize = True))

reduced_data.drop('BoolIncomeVerifiable', axis = 1, inplace = True)

BoolIncomeVerifiable is false:
 Self-employed    0.999616
Not employed     0.000256
Employed         0.000128
Name: StrEmploymentStatus, dtype: float64
BoolIncomeVerifiable is true:
 Employed         0.914816
Other            0.082516
Full-time        0.002592
Self-employed    0.000055
Part-time        0.000021
Name: StrEmploymentStatus, dtype: float64


Last change: Logistic regression requires that the target variable be either 0 or 1, so let's create a new variable called "Cancelled" which is just EnumListingStatus minus 6.

In [21]:
## replace EnumListingStatus with Cancelled
reduced_data['Cancelled'] = reduced_data['EnumListingStatus']-6
reduced_data.drop('EnumListingStatus', axis=1, inplace = True)

In [22]:
## save to pickle file again.
reduced_data.to_pickle('theorem_reduced_secondfilter.pkl')

In [23]:
## What's left:
print(reduced_data.columns.values)

['DolLoanAmountRequested' 'BoolPartialFundingApproved' 'BorrowerRate'
 'NumMonthsTerm' 'EnumListingCategory' 'DolMonthlyIncome'
 'FracDebtToIncomeRatio' 'StrEmploymentStatus' 'StrOccupation'
 'NumMonthsEmployed' 'NumPriorProsperLoans'
 'NumPriorProsperLoansLatePayments' 'NumPriorProsperLoans61dpd'
 'BoolIsLender' 'BoolInGroup' 'EnumChannelCode' 'NumTradesOpened6'
 'NumOpenTradesDelinqOrPastDue6' 'DolTotalBalanceOnPublicRecords'
 'NumRealEstateTrades' 'DolMonthlyDebt' 'NumCurrentDelinquencies'
 'NumPublicRecordsLast10Years' 'NumPublicRecords12' 'DolAmountDelinquent'
 'PctBankcardUtil' 'NumTotalInquiries' 'BoolEverWholeLoan'
 'DaysSinceFirstCredit' 'Cancelled']


This is the final set of variables that we're going to start investigating. We've gone from 86 columns down to 31 columns. Here's a breakdown of what remains:

* Target: Cancelled
* Numerical variables: DolLoanAmountRequested, BorrowerRate, NumMonthsTerm, DolMonthlyIncome, FracDebtToIncomeRatio, NumMonthsEmployed, NumPriorProsperLoans, NumPriorProsperLoansLatePayments, NumPriorProsperLoans61dpd, NumTradesOpened6, NumOpenTradesDelinqOrPastDue6, DolTotalBalanceOnPublicRecords, NumRealEstateTrades, DolMonthlyDebt, NumCurrentDelinquencies, NumPublicRecordsLast10Years, NumPublicRecords12, DolAmountDelinquent, PctBankcardUtil, NumTotalInquiries, DaysSinceFirstCredit (21)
* Categorical variables: EnumListingCategory, StrEmploymentStatus, StrOccupation, EnumChannelCode (4)
* Boolean variables: BoolPartialFundingApproved, BoolIsLender, BoolInGroup, BoolEverWholeLoan (5)

Many of these variables have strange distributions, and will have to be treated individually. In particular, some of these variables are likely better suited to be bools, as demonstrated shortly, and others have large numbers of NaNs that we'll have to figure out what to do with.

Time to visualize the data! NumPriorProsperLoans61dpd is left out because, on closer inspection, all but 50 of the rows have NaNs in that variable. We also use NumPublicRecordsLast10Years instead of NumPublicRecords12 to explore link between public records and cancellation.

In [24]:
## only 50 people have a non-zero value for this criterion!
sum(np.isnan(reduced_data['NumPriorProsperLoans61dpd']))
reduced_data['NumPriorProsperLoans61dpd'].value_counts()

## similarly, very few people have a public record from the last 12 months - let's use 10-year version instead for now
sum(np.isnan(reduced_data['NumPublicRecords12']))
reduced_data['NumPublicRecords12'].value_counts()

0    251457
1       894
2        92
3        18
4         7
6         1
Name: NumPublicRecords12, dtype: int64

In [25]:
## 19 numerical variables only - show plot for each one. Also include column for cancellation.
numerics = reduced_data[['DolLoanAmountRequested','BorrowerRate','NumMonthsTerm','DolMonthlyIncome','FracDebtToIncomeRatio',\
'NumMonthsEmployed','NumPriorProsperLoans','NumPriorProsperLoansLatePayments',\
'NumTradesOpened6','NumOpenTradesDelinqOrPastDue6','DolTotalBalanceOnPublicRecords','NumRealEstateTrades',\
'DolMonthlyDebt','NumCurrentDelinquencies','NumPublicRecordsLast10Years','DolAmountDelinquent','PctBankcardUtil',\
'NumTotalInquiries','DaysSinceFirstCredit']]

In [26]:
## create one big figure behind subpanels
fig, axes = plt.subplots(6,3,figsize=(12,16), facecolor='w')

## cycle through each numeric variable and logistic regress with cancellation (excluding cancellation itself)
for ax, var in zip(axes.reshape(-1),numerics.columns.values):
    sns.regplot(ax=ax, x=var, y="Cancelled", data=reduced_data, logistic=True, ci = None, x_bins = 200)

<IPython.core.display.Javascript object>

* Variables that immediately appear to show a strong correlation with loan cancellation: DolLoanAmountRequested, DolMonthlyIncome,FracDebtToIncomeRatio,NumMonthsEmployed,NumTradesOpened6, NumOpenTradesDelinqOrPastDue6, NumRealEstateTrades, DolMonthlyDebt, NumCurrentDelinquencies, PctBankcardUtil, NumTotalInquiries.

* A few variables immediately seem to demonstrate nonlinear behavior, most notably DaysSinceFirstCredit, DolMonthlyDebt, NumMonthsEmployed and maybe even BorrowerRate. That said, I'm reluctant to add non-linear terms to my eventual model for risk of overfitting.

* NumPriorProsperLoans is a great predictor of whether you cancel or not. This makes some intuitive sense - if you've been through the process before, much easier to follow through. In fact, a boolean variable seems more appropriate, which we implement immediately below.

In [27]:
fig, ax = plt.subplots()
sns.regplot(x="NumPriorProsperLoans", y="Cancelled", data=reduced_data, logistic=True, ci = None, x_bins = 500)

<IPython.core.display.Javascript object>

There's no evidence that taking out more than one loans actually makes you less likely to cancel, so a logistic regression of the number of loans is a poor model. Let's change this variable to a boolean parameter instead.

In [28]:
#Create new Boolean variable that shows whether someone has ever taken out a Prosper loan before or not.
reduced_data['BoolPriorProsperLoanee'] = reduced_data['NumPriorProsperLoans'] > 0
reduced_data.drop('NumPriorProsperLoans', inplace = True, axis =1)

As we did above for the numerical variables, let's check out the influence of the boolean variables in our set.

In [29]:
## 5 boolean variables to consider.
booleans = reduced_data[['BoolPriorProsperLoanee','BoolPartialFundingApproved','BoolIsLender','BoolInGroup','BoolEverWholeLoan']]

In [30]:
## create one big figure behind subpanels
fig, axes = plt.subplots(3,2,figsize=(8,10), facecolor='w')

## cycle through each numeric variable and logistic regress with cancellation (excluding cancellation itself)
for ax, var in zip(axes.reshape(-1), booleans.columns.values):
    sns.regplot(ax=ax, x=var, y="Cancelled", data=reduced_data, logistic=True, ci = None, x_bins = 500)

<IPython.core.display.Javascript object>

BoolPriorProsperLoanee is highly significant, BoolIsLender and BoolInGroup also look significant, 
BoolPartialFundingApproved and BoolEverWholeLoan do not. We drop the latter two variables right away.

In [32]:
reduced_data.drop(['BoolPartialFundingApproved','BoolEverWholeLoan'], inplace = True, axis =1)

In [33]:
print(reduced_data['BoolPriorProsperLoanee'].value_counts(normalize = True))
print(reduced_data['BoolIsLender'].value_counts(normalize = True))
print(reduced_data['BoolInGroup'].value_counts(normalize = True))

False    0.935481
True     0.064519
Name: BoolPriorProsperLoanee, dtype: float64
0    0.987567
1    0.012433
Name: BoolIsLender, dtype: float64
False    0.997366
True     0.002634
Name: BoolInGroup, dtype: float64


Even though these variables all looked significant, BoolIsLender and BoolInGroup are only true for a very small subset of the total loan population - therefore including them into our model probably won't improve skill very much.

In [34]:
reduced_data.to_pickle('theorem_model_variables_test.pkl')

We added a couple of variables and dropped others based on our findings above - let's save our final variable pool.

The plot above gives us a good idea of a few variables that make a big difference, including DolLoanAmountRequested, BoolPriorProsperLoanee and PctBankcardUtil. Loanees are more likely to cancel if the amount of the loan is higher, less likely to cancel if they have received a Prosper loan before, and less likely to cancel with higher bank card utilization. Let's plot some ROC curves for these variables, and also test out another variable, NumTotalInquiries.

In [ ]:
variables = [['DolLoanAmountRequested'],['BoolPriorProsperLoanee','DolLoanAmountRequested'],\
       ['BoolPriorProsperLoanee','DolLoanAmountRequested','PctBankcardUtil'],\
            ['BoolPriorProsperLoanee','DolLoanAmountRequested','PctBankcardUtil','NumTotalInquiries']]

y = reduced_data['Cancelled']
X = []
X_train = []
X_test = []
y_train = []
y_test = []

fig, ax = plt.subplots()
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')

for i,v in enumerate(variables):
    standard_scaler = preprocessing.StandardScaler()
    X.append(standard_scaler.fit_transform(reduced_data[v]))
    X_tr, X_te, y_tr, y_te = train_test_split(X[i], y, test_size=0.3, random_state=0)
    logm = linear_model.LogisticRegression()
    logm.fit(X_tr,y_tr)
    probs = logm.predict_proba(X_te)
    #print(probs[:20])
    fpr, tpr, thresholds = metrics.roc_curve(y_te,probs[:,1])
    plt.plot(fpr,tpr)
    plt.legend()
    print(logm.coef_)

This series of ROC curves shows that starting with DolLoanAmountRequested, then adding BoolPriorProsperLoanee, then PctBankcardUtil, all make successive contributions to quality of fit.

On the other hand, adding NumTotalInquiries makes a relatively minor impact on the resulting ROC curve, so we shoudl consider leaving it out.

Let's figure out how to deal with the categorical variable StrEmploymentStatus.

In [ ]:
## important step...let's test the treatment of the categorical variable.
reduced_data['StrEmploymentStatus'].replace(['Part-time','Not employed'],'Other',inplace = True)
X_test = reduced_data['StrEmploymentStatus'].to_frame()
X_dummies = pd.get_dummies(X_test)

m_dummies = linear_model.LogisticRegression()
m_dummies.fit(X_dummies,y)
print(m_dummies.coef_)

Adding job type notably increases area under curve, especially at the fringes - again, small sub-populations (like
those who respond 'other' as job status) where we're able to make much stronger predictions.